In [1]:
import concurrent.futures
import json
import os
import numpy as np
import pandas as pd
import pickle
import re
import requests

from urllib.parse import urljoin
from bs4 import BeautifulSoup
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # jupyter notebook full-width display
display(HTML("<style>.dataframe td { white-space: nowrap; }</style>")) # no text wrapping

# pandas formatting
pd.set_option('display.float_format', '{:.1f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 200)

# import website url data

In [2]:
links_folder = "website_reports"

dataframes = []

for file in os.listdir(links_folder):
    if file.endswith(".xlsx"):
        file_path = os.path.join(links_folder, file)
        df = pd.read_excel(file_path)
        dataframes.append(df)

column_names = ['type', 'year', 'pub_number', '_', 'nom', 'name', 'url_fr', 'url_en', '_', '_', '_', '_']
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.columns = column_names

types = ['RES', 'SAR', 'PRO', 'SSR', 'SCR', 'ESR', 'HSR']
combined_df = combined_df[combined_df.type.isin(types)]

# create formatted pub number with type and number
combined_df['pub_number'] = combined_df['type'] + " " + combined_df['pub_number']

columns = ['pub_number', 'year', 'nom', 'name', 'url_fr', 'url_en']
combined_df = combined_df[columns].reset_index(drop=True)

In [3]:
# is pub_number distinct in combined_df? yesish - there are 3 errors
combined_df['pub_number'].value_counts().head()

pub_number
SCR 2020/018      2
SSR 2002/D2-01    2
PRO 2024/041      2
ESR 2003/002      1
RES 2021/017      1
Name: count, dtype: int64

In [4]:
# populate pdf filenames where they exist

combined_df['filename_fr'] = combined_df['url_fr'].str.split('/').str[-1]
combined_df['filename_en'] = combined_df['url_en'].str.split('/').str[-1]
combined_df.loc[~combined_df['filename_fr'].str.endswith('.pdf'), 'filename_fr'] = None
combined_df.loc[~combined_df['filename_en'].str.endswith('.pdf'), 'filename_en'] = None

# add file_url columns
combined_df['file_url_fr'] = np.where(combined_df['filename_fr'], combined_df['url_fr'], None)
combined_df['file_url_en'] = np.where(combined_df['filename_en'], combined_df['url_en'], None)

# import parsed publication url data

In [5]:
parsed_docs_folder = os.path.join("..", "ParsedPublications")
min_year, max_year = 1977, 2024
data = []

def process_file(json_path):
    with open(json_path, 'r', encoding='utf-8') as file:
        json_data = json.load(file)
        return {
            'filename': json_data.get('name'),
            'year': json_data.get('publicationYear'),
            'url': json_data.get('url'),
        }

def process_folder(year_path):
    file_data = []
    for json_file in os.listdir(year_path):
        if json_file.endswith(".json"):
            json_path = os.path.join(year_path, json_file)
            file_data.append(process_file(json_path))
    return file_data

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for year_folder in os.listdir(parsed_docs_folder):
        if year_folder.isnumeric() and min_year <= int(year_folder) <= max_year:
            year_path = os.path.join(parsed_docs_folder, year_folder)
            if os.path.isdir(year_path):
                futures.append(executor.submit(process_folder, year_path))

    for future in concurrent.futures.as_completed(futures):
        data.extend(future.result())

parsed_docs_df = pd.DataFrame(data) # this took 3 seconds
unmatched_url = set(parsed_docs_df['url'].to_list())

# confirm all url are distinct
parsed_docs_df['url'].value_counts().value_counts()

count
1    12752
Name: count, dtype: int64

# STEP 1. crawl bs4 to get remaining filename_fr and filename_en
* this uses unmatched_url from parsed data json files - this turns out to be a mistake because of spaces, formatting etc. error data is logged and used to import in the next step

In [6]:
# what are the different link suffices? -> 'html', 'htm', 'pdf' 
#  exclude pdf or not htm/html
display(combined_df['url_en'].str.endswith(('html', 'htm')).value_counts())
display(combined_df['url_en'].str.endswith('pdf').value_counts())
display(combined_df['url_fr'].str.endswith(('html', 'htm')).value_counts())
display(combined_df['url_fr'].str.endswith('pdf').value_counts())

url_en
True     7692
False    1369
Name: count, dtype: int64

url_en
False    7692
True     1369
Name: count, dtype: int64

url_fr
True     7692
False    1369
Name: count, dtype: int64

url_fr
False    7692
True     1369
Name: count, dtype: int64

In [7]:
errors = dict()


def find_pdf_link(url):
    global errors
    
    if url.split('.')[-1].lower() not in ['html', 'htm']:
        return None
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        pdf_links = re.findall(r'http[s]?://[^\'"<>]+\.pdf', str(soup), re.IGNORECASE)
        
        for link in pdf_links:
            if link in unmatched_url:
                return link
            
        errors[url] = f"pdf link not found. links on website: {pdf_links=}"
                
    except requests.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        errors[url] = f"find_pdf_link Error: {e}"
        
    return None

In [8]:
# conditionally load combined_df from csv if it exists

output_file = 'temp/updated_parsed_docs.csv'


if os.path.exists(output_file):
    df = pd.read_csv(output_file)
    print("File loaded successfully.")
else:
    print(f"{output_file} does not exist.")

updated_parsed_docs.csv does not exist.


In [9]:
batch_size = 100
save_batch = True


for index, row in combined_df.iterrows():
    if save_batch:
        if index % batch_size == 0 and index > 0:
            combined_df.to_csv(output_file, index=False)
            print(f"Progress saved after {index} rows.")
    
    if (row['filename_fr'] and row['filename_en']) or (row['file_url_fr'] and row['file_url_en']):
        continue
    
    pdf_link_fr = None
    pdf_link_en = None
    pub_number = row['pub_number']  # should be distinct for error tracking (there are 3 duplicates / errors)
    
    if pd.isna(row['filename_fr']):
        pdf_link_fr = find_pdf_link(row['url_fr'])
        if pdf_link_fr:
            combined_df.at[index, 'file_url_fr'] = pdf_link_fr
            if pdf_link_fr not in unmatched_url:
                errors[pub_number + ' (fr)'] = 'Link was added to combined_df but was not in unmatched_url (fr)'
        else:
            errors[pub_number + ' (fr)'] = 'pdf_link_fr was not found on webpage (fr)'

    if pd.isna(row['filename_en']):
        if row['url_en'] != row['url_fr']:
            pdf_link_en = find_pdf_link(row['url_en'])
            if pdf_link_en:
                combined_df.at[index, 'file_url_en'] = pdf_link_en
                if pdf_link_en not in unmatched_url:
                    errors[pub_number + ' (en)'] = 'Link was added to combined_df but was not in unmatched_url (en)'
            else:
                errors[pub_number + ' (en)'] = 'pdf_link_en was not found on webpage (en)'
        else:
            if pdf_link_fr:
                combined_df.at[index, 'file_url_en'] = pdf_link_fr
                if pdf_link_fr not in unmatched_url:
                    errors[pub_number + ' (en)'] = 'Link was added to combined_df but was not in unmatched_url (en)'
            else:
                errors[pub_number + ' (en)'] = 'pdf_link_fr was not found on webpage (en)'


combined_df.to_csv(output_file, index=False)

with open('errors.pickle', 'wb') as f:
    pickle.dump(errors, f)


Progress saved after 100 rows.
Progress saved after 200 rows.
Progress saved after 300 rows.
Progress saved after 400 rows.
Progress saved after 500 rows.
Progress saved after 600 rows.
Progress saved after 700 rows.
Progress saved after 800 rows.
Progress saved after 900 rows.
Progress saved after 1000 rows.
Progress saved after 1100 rows.
Progress saved after 1200 rows.
Progress saved after 1300 rows.
Progress saved after 1400 rows.
Progress saved after 1500 rows.
Progress saved after 1600 rows.
Progress saved after 1700 rows.
Progress saved after 1800 rows.
Progress saved after 1900 rows.
Progress saved after 2000 rows.
Progress saved after 2100 rows.
Progress saved after 2200 rows.
Progress saved after 2300 rows.
Progress saved after 2400 rows.
Progress saved after 2500 rows.
Progress saved after 2600 rows.
Progress saved after 2700 rows.
Progress saved after 2800 rows.
Progress saved after 2900 rows.
Progress saved after 3000 rows.
Progress saved after 3100 rows.
Progress saved af

# Error checkin and fixin

In [42]:
# conditionally load combined_df from csv if it exists

output_file = 'updated_parsed_docs.csv'


if os.path.exists(output_file):
    combined_df = pd.read_csv(output_file)
    print("File loaded successfully.")
else:
    print(f"{output_file} does not exist.")

File loaded successfully.


In [13]:
errors

{'http://www.dfo-mpo.gc.ca/csas-sccs/publications/withdrawn-retire-fra.htm': 'pdf link not found. links on website: pdf_links=[]',
 'PRO 2000/006 (fr)': 'pdf_link_fr was not found on webpage (fr)',
 'http://www.dfo-mpo.gc.ca/csas-sccs/publications/withdrawn-retire-eng.htm': 'pdf link not found. links on website: pdf_links=[]',
 'PRO 2000/006 (en)': 'pdf_link_en was not found on webpage (en)',
 'PRO 2001/028 (fr)': 'pdf_link_fr was not found on webpage (fr)',
 'PRO 2001/028 (en)': 'pdf_link_en was not found on webpage (en)',
 'PRO 2001/001 (fr)': 'pdf_link_fr was not found on webpage (fr)',
 'PRO 2001/001 (en)': 'pdf_link_en was not found on webpage (en)',
 'PRO 2004/024 (fr)': 'pdf_link_fr was not found on webpage (fr)',
 'PRO 2004/024 (en)': 'pdf_link_en was not found on webpage (en)',
 'PRO 2005/023 (fr)': 'pdf_link_fr was not found on webpage (fr)',
 'PRO 2005/023 (en)': 'pdf_link_en was not found on webpage (en)',
 'PRO 2005/021 (fr)': 'pdf_link_fr was not found on webpage (fr)',
 

### manually fix 1 typo in url

In [44]:
error_in_url = 'http://www.dfo-mpo.gcca/csas-sccs/Publications/ResDocs-DocRech/2021/2021_033-fra.html'  # needs a period - fix manually
combined_df.iloc[6072]

pub_number                                                                                                                                                                                                RES 2021/033
year                                                                                                                                                                                                              2021
nom            Renseignements à l'appui d'une évaluation du potentiel de rétablissement de l'esturgeon jaune, Acipenser fulvescens (populations de l'ouest de la baie d'Hudson, de la rivière Saskatchewan et du fl...
name                  Information in support of a recovery potential assessment of Lake Sturgeon, Acipenser fulvescens (Western Hudson Bay, Saskatchewan-Nelson River, and Great Lakes-Upper St. Lawrence populations)
url_fr                                                                                                                           http://www.

In [45]:
combined_df.loc[combined_df.url_fr == error_in_url, 'url_fr'] = combined_df.loc[combined_df.url_fr == error_in_url, 'url_fr'].str.replace('dfo-mpo.gcca', 'dfo-mpo.gc.ca')
combined_df.iloc[6072]

pub_number                                                                                                                                                                                                RES 2021/033
year                                                                                                                                                                                                              2021
nom            Renseignements à l'appui d'une évaluation du potentiel de rétablissement de l'esturgeon jaune, Acipenser fulvescens (populations de l'ouest de la baie d'Hudson, de la rivière Saskatchewan et du fl...
name                  Information in support of a recovery potential assessment of Lake Sturgeon, Acipenser fulvescens (Western Hudson Bay, Saskatchewan-Nelson River, and Great Lakes-Upper St. Lawrence populations)
url_fr                                                                                                                          http://www.d

### fix missing url with pdf_links

In [46]:
# example of missing
'https://waves-vagues.dfo-mpo.gc.ca/library-bibliotheque/279332.pdf' in unmatched_url

False

In [49]:
'279332.pdf' in parsed_docs_df.filename.to_list()

True

In [50]:
parsed_docs_df[parsed_docs_df.filename == '279332.pdf']

,filename,year,url
1843,279332.pdf,2003,https://waves-vagues.dfo-mpo.gc.ca/library-bibliotheque/279332.pdf


In [52]:
str(parsed_docs_df[parsed_docs_df.filename == '279332.pdf'].url) == 'https://waves-vagues.dfo-mpo.gc.ca/library-bibliotheque/279332.pdf'

False

In [53]:
'https://waves-vagues.dfo-mpo.gc.ca/library-bibliotheque/279332.pdf' == 'https://waves-vagues.dfo-mpo.gc.ca/library-bibliotheque/279332.pdf'

True

In [54]:
# there is a space ....... ugggh
[x for x in unmatched_url if x.split('/')[-1] == '279332.pdf']

[' https://waves-vagues.dfo-mpo.gc.ca/library-bibliotheque/279332.pdf']

In [61]:
pdf_links_from_errors = dict()

for k, v in errors.items():
    if v.endswith(']') and not v.endswith('[]'):
        html_link = k
        pdf_link = v.split('[')[-1].split(']')[0].replace("'", "").replace('"', '')
        pdf_links_from_errors[html_link] = pdf_link

In [63]:
for link in pdf_links_from_errors.values():
    if not link.endswith('pdf'):
        print(f'not a pdf: {link}')

# STEP 2: loop through errors to get url that weren't in unmatched_url (as noted above) 

In [137]:
new_errors = dict()
n = 0

for index, row in combined_df.iterrows():
    if pd.notna(row['file_url_fr']) and pd.notna(row['file_url_en']):
        continue
    
    file_url_fr = None
    file_url_en = None
    pub_number = row['pub_number']
    
    if pd.isna(row['file_url_fr']):
        file_url_fr = pdf_links_from_errors.get(row['url_fr'])
        if file_url_fr:
            combined_df.at[index, 'file_url_fr'] = file_url_fr
            n += 1
        else:
            new_errors[pub_number + ' (fr)'] = 'Link not found in errors list (fr)'
    
    if pd.isna(row['file_url_en']):
        file_url_en = pdf_links_from_errors.get(row['url_en'])
        if file_url_en:
            combined_df.at[index, 'file_url_en'] = file_url_en
            n += 1
        else:
            new_errors[pub_number + ' (en)'] = 'Link not found in errors list (en)'

combined_df.to_csv(output_file, index=False)

# what is still missing? withdrawn and relative url

In [139]:
# 137 pdf url still missing
len(new_errors)

137

In [ ]:
still_missing_df = combined_df[(combined_df['file_url_fr'].isnull() | combined_df['file_url_en'].isnull()) & ~(combined_df['url_fr'].str.contains('withdrawn'))].copy()
url_still_missing = still_missing_df.url_en.to_list() + still_missing_df.url_fr.to_list()

In [189]:
final_errors = dict()
soup_dict = dict()


def updated_find_pdf_link(url):
    global final_errors, soup_dict

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        pdf_links = [
            urljoin(url, a['href'])
            for a in soup.find_all('a', href=True)
            if a['href'].strip().lower().endswith('.pdf')
        ]

        if pdf_links:
            return pdf_links[0]

        if "withdrawn" in soup.get_text().lower():
            return "WITHDRAWN"

        final_errors[url] = f"PDF link not found. Links on website: {pdf_links}"
        soup_dict[url] = soup

    except requests.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        final_errors[url] = f"find_pdf_link Error: {e}"

    return None

In [190]:
final_links = dict()

for link in url_still_missing:
    updated_link = updated_find_pdf_link(link)
    if updated_link:
        final_links[link] = updated_link


# STEP 3: loop through url_still_missing and crawl website for relative href or 'withdrawn' status

In [193]:
for index, row in combined_df.iterrows():
    if pd.notna(row['file_url_fr']) and pd.notna(row['file_url_en']):
        continue
    
    file_url_fr = None
    file_url_en = None
    pub_number = row['pub_number']
    
    if pd.isna(row['file_url_fr']):
        file_url_fr = final_links.get(row['url_fr'])
        if file_url_fr:
            combined_df.at[index, 'file_url_fr'] = file_url_fr
            n += 1
        else:
            final_errors[pub_number + ' (fr)'] = 'Link not found in final_links list (fr)'
    
    if pd.isna(row['file_url_en']):
        file_url_en = final_links.get(row['url_en'])
        if file_url_en:
            combined_df.at[index, 'file_url_en'] = file_url_en
            n += 1
        else:
            final_errors[pub_number + ' (en)'] = 'Link not found in final_links list (en)'

combined_df.to_csv(output_file, index=False)

In [196]:
# all links accounted for, or withdrawn
combined_df[(combined_df['file_url_fr'].isnull() | combined_df['file_url_en'].isnull()) & ~(combined_df['url_fr'].str.contains('withdrawn'))]

,pub_number,year,nom,name,url_fr,url_en,filename_fr,filename_en,file_url_fr,file_url_en


In [199]:
# add "WITHDRAWN" to links for withdrawn
combined_df.loc[combined_df['url_fr'].str.contains('withdrawn'), ['file_url_fr', 'file_url_en']] = "WITHDRAWN"

In [200]:
# all file url links complete
combined_df[combined_df['file_url_fr'].isnull() | combined_df['file_url_en'].isnull()]

,pub_number,year,nom,name,url_fr,url_en,filename_fr,filename_en,file_url_fr,file_url_en


In [205]:
# add filenames to combined_df
combined_df['pdf_fr'] = combined_df['file_url_fr'].str.split('/').str[-1]
combined_df['pdf_en'] = combined_df['file_url_en'].str.split('/').str[-1]

In [212]:
combined_df[combined_df.pdf_fr != combined_df.filename_fr]['filename_fr'].value_counts(dropna=False)

filename_fr
NaN    7692
Name: count, dtype: int64

In [213]:
combined_df[combined_df.pdf_en != combined_df.filename_en]['filename_en'].value_counts(dropna=False)

filename_en
NaN    7692
Name: count, dtype: int64

# all pdf values match as expected

In [214]:
combined_df.head(1).T

,0
pub_number,ESR 2003/002
year,2003
nom,"État de l'océan en 2001 : Conditions océanographiques physiques sur le plateau néo-écossais, dans la baie de Fundy et dans le golfe du Maine."
name,"2001 State of the Ocean: Physical Oceanographic Conditions on the Scotian Shelf, Bay of Fundy and Gulf of Maine."
url_fr,http://waves-vagues.dfo-mpo.gc.ca/Library/279108.pdf
url_en,http://waves-vagues.dfo-mpo.gc.ca/Library/276322.pdf
filename_fr,279108.pdf
filename_en,276322.pdf
file_url_fr,http://waves-vagues.dfo-mpo.gc.ca/Library/279108.pdf
file_url_en,http://waves-vagues.dfo-mpo.gc.ca/Library/276322.pdf


In [215]:
combined_df['filename_fr'] = combined_df['pdf_fr']
combined_df['filename_en'] = combined_df['pdf_en']

In [219]:
combined_df = combined_df.drop(['pdf_fr', 'pdf_en'], axis=1)

In [221]:
save_final_data = False # already saved, don't save over

if save_final_data:
    combined_df.to_csv("fr_eng_correlation_data.csv", index=False)